In [0]:
from datascience import *
from urllib.request import urlopen
import numpy as np
import pandas as pd
import plotly.express as px




In [1]:
# Read a text file to a dataframe using colon delimiters
import pandas as pd
clinical_samples = pd.read_csv('MSK2018data_clinical_sample.txt', sep='[\n\t]', engine='python') #all the clinical samples (patients and samples)
patients = pd.read_csv('MSK2018data_clinical_patient.txt', sep='[\n\t]', engine='python') #info about the patients only plus metadata
mutations = pd.read_csv("MSK2018data_mutations.txt",sep='[\n\t]', engine='python') #different mutations observed, has a unique transcriptID

#I don't want to use gene_panel or cna, not that great after I attempted once
#all the ones below are datasets I looked at and attempted to see if they could be utilized
#i have decided that they are not
gene_panel = pd.read_csv("MSK2018data_gene_panel_matrix.txt",sep='[\n\t]', engine='python') #sample_id, with mutations id (IMPACT###), CNA id (impact###) and structural variants
cna = pd.read_csv("MSK2018data_cna.txt",sep='[\n\t]', engine='python') #this has all the genes as rows and columns as the sampels
cna = cna.T #i switch it around
cna.index.names = ["Sample ID"]
cna.columns = cna.iloc[0] #set column names equal to values in row index position 0
cna = cna[1:]#remove first row from DataFrame
sv = pd.read_csv("MSK2018data_sv.txt",sep='[\n\t]', engine='python') #sampleID's with hugo_symbols (multiples), not all samples
chromosomes = pd.read_csv("MSK2018data_cna_hg19.txt",sep='[\n\t]', engine='python') #different types of chromosomes, after some looking doesn't seem that important

#make mutations good in terms of column names and data types
mutations['Chromosome'] = mutations['Chromosome'].astype(str)
mutations['Chromosome'] = mutations['Chromosome'].replace('X', '23')
mutations['Chromosome'] = mutations['Chromosome'].astype(str).astype(float)
mutations['Start_Position'] = mutations['Start_Position'].astype(float)
mutations = mutations.drop(['Match_Norm_Seq_Allele1', 'Match_Norm_Seq_Allele2', 'Match_Norm_Validation_Allele1', 'Match_Norm_Validation_Allele2',
                            'Tumor_Validation_Allele1','Tumor_Validation_Allele2', 'Matched_Norm_Sample_Barcode', 'dbSNP_RS', 'dbSNP_Val_Status',
                           'Verification_Status', 'Validation_Status', 'Sequencing_Phase', 'Sequence_Source', 'Validation_Method', 'BAM_File', 'Sequencer'], axis = 1)
                             
                           


In [163]:
x = mutations.groupby(["Hugo_Symbol"]).count()

In [12]:
pd.set_option('display.max_columns', None)

In [195]:
clin_joined = clinical_samples.join(patients.set_index('#Patient Identifier'), on = "#Patient Identifier") #i joined the samples with their patient ID
clin_joined.set_index("Sample Identifier")
mut_joined = clin_joined.join(mutations.set_index("Tumor_Sample_Barcode"), on='Sample Identifier')

In [198]:
mut_joined.head(10)

,#Patient Identifier,Sample Identifier,Sample Type,Site of Sample,Prior Breast Primary,Time to Diagnosis Invasive Carcinoma,NGS Sample Collection Time Period,Invasive Carcinoma Diagnosis Age,Tumor Tissue Origin,Tumor Sample Histology,Overall Primary Tumor Grade,Primary Nuclear Grade,Stage At Diagnosis,Receptor Status Primary,ER Status of the Primary,ER PCT Primary,PR Status of the Primary,PR PCT Primary,HER2 IHC Status Primary,HER2 IHC Score Primary,HER2 FISH Status (Report and ASCO) of Primary,HER2 FISH Ratio Primary,Overall HR Status of Sequenced Sample,ER Status of Sequenced Sample,PR Status of Sequenced Sample,Overall HER2 Status of Sequenced Sample,HER2 IHC Status of Sequenced Sample,HER2 IHC Score of Sequenced Sample,HER2 FISH Status of Sequenced Sample,HER2 FISH Ratio Value of Sequenced Sample,Oncotree Code,Cancer Type,Cancer Type Detailed,Somatic Status,TMB (nonsynonymous),Metastatic Disease at Last Follow-up,Metastatic Recurrence Time,Prior Local Recurrence,Sex,Menopausal Status At Diagnosis,Primary Tumor Laterality,T Stage,N Stage,M Stage,Overall Patient Receptor Status,Overall Patient HR Status,Overall Patient HER2 Status,HER2 Primary Status,Patient's Vital Status,Last Communication Contact,Time To Death (Months),Disease Free (Months),Disease Free Event,Overall Survival (Months),Overall Survival Status,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Mutation_Status,Score,t_ref_count,t_alt_count,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Hotspot
0,#Identifier to uniquely specify a patient.,A unique sample identifier.,"The type of sample (i.e., normal, primary, met...",Sample Site,Prior Breast Primary,Time to Diagnosis Invasive Carcinoma,NGS Sample Collection Time Period,Invasive Carcinoma Diagnosis Age,Tumor Tissue of Origin of the Sequnced Tumor.,Tumor Sample Histology,Overall Primary Tumor Histological Grade.,Primary Nuclear Grade,Stage At Diagnosis,Receptor Status of the Primary Tumor,ER Status of the Primary Tumor,ER Percent Positivity of the Primary Tumor,PR Status of the Primary Tumor,PR Percent Positivity of the Primary Tumor,HER2 IHC Status of Primary Tumor,HER2 IHC Value of the Primary Tumor,HER2 FISH Status of the Primary Tumor as per P...,HER2 FISH Ratio of the Primary Tumor,Overall HR Status of the Sample that Underwent...,ER Status of the Sample that Underwent Sequenc...,PR status of the Sample that Underwent Sequenc...,Overall HER2 Status of the Sample that Underwe...,HER2 IHC Status of the Sample that Underwent S...,HER2 IHC Value of the Sample that Underwent Se...,HER2 FISH Status of the Sample that Underwent ...,HER2 FISH Ratio Value of the Sample that Under...,Oncotree Code,Cancer Type,Cancer Type Detailed,Somatic Status,TMB (nonsynonymous),Indicate metastatic disease at last followup.,Months between diagnosis of the primary tumor ...,Prior Local Recurrence,Sex,Menopausal Status At Diagnosis,"For tumors in paired organs, designates the si...",T Stage,N Stage,M Stage,Overall patient tumor receptor subtype based o...,Overall patient tumor hormone receptor status ...,Overall patient tumor HER2 status based on the...,Overall HER2 status of the primary tumor.,The survival state of the person.,Time of the last follow up in days (reference:...,Time To Death (Months),Disease free (months) since initial treatment.,Disease Free Event,Overall survival in months since initial diago...,Overall patient survival status.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#STRING,STRING,STRING,STRING,STRING,NUMBER,NUMBER,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,ST

In [199]:
mut_joined.to_csv("MSK_2018")

In [20]:
# Read a text file to a dataframe using colon delimiters
import pandas as pd
BRCAclin_samp = pd.read_csv('BRCAdata_clinical_sample.txt', sep='[\n\t]', engine='python') #all the clinical samples (patients and samples)
BRCApatients = pd.read_csv('BRCAdata_clinical_patient.txt', sep='[\n\t]', engine='python') #info about the patients only plus metadata
BRCAmut = pd.read_csv("BRCAdata_mutations.txt",sep='[\n\t]', engine='python') #different mutations observed, has a unique transcriptID
#drop columns that are pure NA
BRCAmut = BRCAmut.drop(['dbSNP_RS', 'dbSNP_Val_Status', 'Matched_Norm_Sample_Barcode', 'Match_Norm_Seq_Allele1', 'Match_Norm_Seq_Allele2', 
                            'Tumor_Validation_Allele1', 'Tumor_Validation_Allele2', 'Match_Norm_Validation_Allele1', 'Match_Norm_Validation_Allele2',
                            'Verification_Status', 'Validation_Status', 'Mutation_Status', 'Sequencing_Phase', 'Sequence_Source', 'Validation_Method',
                            'Score', 'BAM_File', 't_ref_count', 't_alt_count', 'n_ref_count', 'n_alt_count'], axis = 1)
                             


#I don't want to use gene_panel or cna, not that great after I attempted once
#all the ones below are datasets I looked at and attempted to see if they could be utilized
#i have decided that they are not
#gene_panel = pd.read_csv("data_gene_panel_matrix.txt",sep='[\n\t]', engine='python') #sample_id, with mutations id (IMPACT###), CNA id (impact###) and structural variants
#cna = pd.read_csv("data_cna.txt",sep='[\n\t]', engine='python') #this has all the genes as rows and columns as the sampels
#cna = cna.T #i switch it around
#cna.index.names = ["Sample ID"]
#cna.columns = cna.iloc[0] #set column names equal to values in row index position 0
#cna = cna[1:]#remove first row from DataFrame
#sv = pd.read_csv("data_sv.txt",sep='[\n\t]', engine='python') #sampleID's with hugo_symbols (multiples), not all samples
#chromosomes = pd.read_csv("data_cna_hg19.txt",sep='[\n\t]', engine='python') #different types of chromosomes, after some looking doesn't seem that important


In [25]:
BRCAclin_joined = BRCAclin_samp.join(BRCApatients.set_index("#Patient Identifier"), on = "#Patient Identifier")
BRCAmut_joined = BRCAclin_joined.join(BRCAmut.set_index("Tumor_Sample_Barcode"), on='Sample Identifier')

In [26]:
BRCAmut_joined

,#Patient Identifier,Sample Identifier,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage,TMB (nonsynonymous),Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Sex,Integrative Cluster,Age at Diagnosis,Overall Survival (Months),Overall Survival Status,Pam50 + Claudin-low subtype,3-Gene classifier subtype,Patient's Vital Status,Primary Tumor Laterality,Radio Therapy,Tumor Other Histologic Subtype,Type of Breast Surgery,Relapse Free Status,Relapse Free Status (Months),Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Sequencer,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Hotspot
0,#Identifier to uniquely specify a patient.,A unique sample identifier.,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Numeric value to express the degree of abnorma...,Oncotree Code,PR Status,"The type of sample (i.e., normal, primary, met...",Tumor size.,Tumor stage.,TMB (nonsynonymous),Number of lymphnodes positive,Nottingham prognostic index,Tumor Content,Chemotherapy.,Cohort.,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Sex,Integrative Cluster,Age at Diagnosis,Overall survival in months since initial diago...,Overall patient survival status.,Pam50 + Claudin-low subtype,3-Gene classifier subtype,The survival state of the person.,"For tumors in paired organs, designates the si...",Radio Therapy,Text to describe a tumor's histologic subtype ...,Type of Breast Surgery,Relapse Free Status (Recurred = cases where th...,Time until last follow-up or relapse (in months),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,ER_STATUS,HER2_STATUS,GRADE,ONCOTREE_CODE,PR_STATUS,SAMPLE_TYPE,TUMOR_SIZE,TUMOR_STAGE,TMB_NONSYNONYMOUS,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,SEX,INTCLUST,AGE_AT_DIAGNOSIS,OS_MONTHS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY,RFS_STATUS,RFS_MONTHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MB-0000,MB-0000,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Negative,Primary,22,2,0,10,6.044,NaN,NO,1,Positve,NEUTRAL,YES,Post,Female,4ER+,75.65,140.5,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY,0:Not Recurred,138.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,MTS-T2432,MTS-T2432,Breast Cancer,Invasive Breast Carcinoma,NaN,NaN,NaN,BRCA,NaN,Primary,NaN,0,6.537588519,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,None,None,None,None,None,None,None,None,None,None,None,None,None,THSD7A,NaN,METABRIC,GRCh37,7,11418906.0,11418906.0,+,"missense_variant,splice_region_variant",Missense_Muta

In [27]:
BRCAclin_joined

,#Patient Identifier,Sample Identifier,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage,TMB (nonsynonymous),Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Sex,Integrative Cluster,Age at Diagnosis,Overall Survival (Months),Overall Survival Status,Pam50 + Claudin-low subtype,3-Gene classifier subtype,Patient's Vital Status,Primary Tumor Laterality,Radio Therapy,Tumor Other Histologic Subtype,Type of Breast Surgery,Relapse Free Status,Relapse Free Status (Months)
0,#Identifier to uniquely specify a patient.,A unique sample identifier.,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Numeric value to express the degree of abnorma...,Oncotree Code,PR Status,"The type of sample (i.e., normal, primary, met...",Tumor size.,Tumor stage.,TMB (nonsynonymous),Number of lymphnodes positive,Nottingham prognostic index,Tumor Content,Chemotherapy.,Cohort.,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Sex,Integrative Cluster,Age at Diagnosis,Overall survival in months since initial diago...,Overall patient survival status.,Pam50 + Claudin-low subtype,3-Gene classifier subtype,The survival state of the person.,"For tumors in paired organs, designates the si...",Radio Therapy,Text to describe a tumor's histologic subtype ...,Type of Breast Surgery,Relapse Free Status (Recurred = cases where th...,Time until last follow-up or relapse (in months)
1,#STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,STRING,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER,NUMBER,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER
2,#1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,ER_STATUS,HER2_STATUS,GRADE,ONCOTREE_CODE,PR_STATUS,SAMPLE_TYPE,TUMOR_SIZE,TUMOR_STAGE,TMB_NONSYNONYMOUS,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,SEX,INTCLUST,AGE_AT_DIAGNOSIS,OS_MONTHS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY,RFS_STATUS,RFS_MONTHS
4,MB-0000,MB-0000,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Negative,Primary,22,2,0,10,6.044,NaN,NO,1,Positve,NEUTRAL,YES,Post,Female,4ER+,75.65,140.5,0:LIVING,claudin-low,ER-/HER2-,Living,Right,YES,Ductal/NST,MASTECTOMY,0:Not Recurred,138.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,MTS-T2428,MTS-T2428,Breast Cancer,Invasive Breast Carcinoma,Positive,NaN,1,BRCA,NaN,Primary,27,1,2.615035408,0,2.54,NaN,NaN,1,Positve,NaN,NaN,NaN,Female,NaN,70.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:Recurred,4.93
2509,MTS-T2429,MTS-T2429,Breast Cancer,Invasive Breast Carcinoma,Positive,NaN,2,BRCA,NaN,Primary,28,2,5.230070815,0,4.56,NaN,NaN,1,Positve,NaN,NaN,NaN,Female,NaN,63.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:Recurred,16.18
2510,MTS-T2430,MTS-T2430,Breast Cancer,Invasive Breast Carcinoma,NaN,NaN,NaN,BRCA,NaN,Primary,NaN,0,7.845106223,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2511,MTS-T2431,MTS-T2431,Breast Cancer,Invasive Breast Carcinoma,NaN,NaN,NaN,BRCA,NaN,Primary,NaN,0,9.152623927,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [28]:
BRCAclin_samp

,#Patient Identifier,Sample Identifier,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage,TMB (nonsynonymous)
0,#Identifier to uniquely specify a patient.,A unique sample identifier.,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Numeric value to express the degree of abnorma...,Oncotree Code,PR Status,"The type of sample (i.e., normal, primary, met...",Tumor size.,Tumor stage.,TMB (nonsynonymous)
1,#STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,STRING,NUMBER
2,#1,1,1,1,1,1,1,1,1,1,1,1,1
3,PATIENT_ID,SAMPLE_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,ER_STATUS,HER2_STATUS,GRADE,ONCOTREE_CODE,PR_STATUS,SAMPLE_TYPE,TUMOR_SIZE,TUMOR_STAGE,TMB_NONSYNONYMOUS
4,MB-0000,MB-0000,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Negative,Primary,22,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,MTS-T2428,MTS-T2428,Breast Cancer,Invasive Breast Carcinoma,Positive,NaN,1,BRCA,NaN,Primary,27,1,2.615035408
2509,MTS-T2429,MTS-T2429,Breast Cancer,Invasive Breast Carcinoma,Positive,NaN,2,BRCA,NaN,Primary,28,2,5.230070815
2510,MTS-T2430,MTS-T2430,Breast Cancer,Invasive Breast Carcinoma,NaN,NaN,NaN,BRCA,NaN,Primary,NaN,0,7.845106223
2511,MTS-T2431,MTS-T2431,Breast Cancer,Invasive Breast Carcinoma,NaN,NaN,NaN,BRCA,NaN,Primary,NaN,0,9.152623927
